In [21]:
try:
    import requests
    import os
    from langchain_google_genai import ChatGoogleGenerativeAI
    from dotenv import load_dotenv
    from langchain.output_parsers import PydanticOutputParser
    from langchain.prompts import PromptTemplate
    from pydantic import BaseModel, Field
    from typing import List
except ImportError:
    !pip3 install requests python-dotenv langchain-google-genai "langchain>0.3.0,<1.0.0" pydantic
    import requests
    import os
    from langchain_google_genai import ChatGoogleGenerativeAI
    from dotenv import load_dotenv
    from langchain.output_parsers import PydanticOutputParser
    from langchain.prompts import PromptTemplate
    from pydantic import BaseModel, Field
    from typing import List

In [17]:
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

In [23]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.7,
    api_key=api_key
)

E0000 00:00:1770579972.060025 6243328 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [26]:
class DayItinerary(BaseModel):
  day: int = Field(description="Day number of the trip")
  activities: list[str] = Field(description="List of activities planned for the day. Keep it short and concise")
  daily_budget: int = Field(description="Estimated budget for the day")

class TravelPlan(BaseModel):
  destination: str = Field(description="Travel destination: City, Country")
  reasoning: str = Field(description="Reason for choosing this destination. 1-2 setences")
  duration: int = Field(description="Duration of the trip in days")
  total_budget: int = Field(description = "Total budget for the trip")
  estimated_cost: int = Field(description="Estimated total cost of the trip based on the activities planned, hotel accomadations, and transport costs")
  itinerary: list[DayItinerary] = Field(description="A day by day itinerary for the trip. Each day should have around 3 activities planned. Keep it short and concise")

parser = PydanticOutputParser(pydantic_object=TravelPlan)

prompt = PromptTemplate(
template="""
  You are a travel planner. Your task is to create a travel plan for a trip based on the user's preferences. 
  Take into account the user's budget: {budget} CAD, trip length: {duration} days, travel month: {month}, 
  and interests/hobbies: {interests}. 
  When generating the travel plan, ensure that the total estimated cost of the trip including transportation, accommodation, 
  and activities does not exceed the user budget of: {budget} CAD. 

  {format_instructions}
  """,
  input_variables=["budget", "duration", "month", "interests"],
  partial_variables={"format_instructions": parser.get_format_instructions()}
)

response = llm.invoke(prompt.format(
  budget = 2000,
  duration = 10,
  month = "June",
  interests = ["hiking", "food", "culture"]
))

parsed_response = parser.parse(response.content)

print("structured response:")
print(f"destination: {parsed_response.destination}")
print(f"reasoning: {parsed_response.reasoning}")
print(f"duration: {parsed_response.duration} days")
print(f"total_budget: {parsed_response.total_budget} CAD")
print(f"estimated_cost: {parsed_response.estimated_cost} CAD")
print("itinerary:")
for day in parsed_response.itinerary:
  print(f"  Day {day.day}:")
  print(f"    activities: {', '.join(day.activities)}")
  print(f"    daily_budget: {day.daily_budget} CAD")  


structured response:
destination: Southern Ontario & Quebec, Canada
reasoning: This road trip offers a perfect blend of hiking in provincial parks, exploring the rich culture and history of major cities, and indulging in diverse food experiences, all within the specified budget for June.
duration: 10 days
total_budget: 2000 CAD
estimated_cost: 1970 CAD
itinerary:
  Day 1:
    activities: Drive from Toronto to Algonquin Provincial Park (approx. 3-4 hours), Check into budget accommodation near the park (e.g., motel, cabin), Easy hike: Spruce Bog Boardwalk
    daily_budget: 195 CAD
  Day 2:
    activities: Morning hike: Track and Tower Trail (moderate), Afternoon canoeing on a lake in Algonquin Park, Stargazing
    daily_budget: 165 CAD
  Day 3:
    activities: Short morning hike: Booth's Rock Trail for scenic views, Drive from Algonquin Park to Ottawa (approx. 3-4 hours), Explore the vibrant ByWard Market and grab dinner
    daily_budget: 180 CAD
  Day 4:
    activities: Visit Parliament